## Evaluating the system performance

This notebook evaluates the performance of the system by comparing the ranked list of pieces with the ground truth for each query.

In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import glob
import os.path
import pickle
import random

In [ ]:
def readGroundTruthLabels(gtfile):
    d={}
    d1={}
    with open(gtfile, "r") as f:
        for line in f:
            data = line.split(',')
            key = data[0]
            d[key] = []
            d1[key] = key
            for idx,item in enumerate(data):
                if idx != 0 and item != "x":
                    try:
                        pieceNum = int(item)
                        d[key].append(pieceNum)
                    except:
                        pass
                    try:
                        pieceNum = int(item)
                        d1[str(pieceNum)] = key
                    except:
                        pass
    return d, d1

In [ ]:
def readHypothesisFiles(hypdir, benchmark):
    l = []
    for hypfile in sorted(glob.glob("{}/*.hyp".format(hypdir))):
#         print(hypfile)
        with open(hypfile, "rb") as f:
            l.append(pickle.load(f))
    return l

In [ ]:
s = set([])
def collapseIds(pieceScores):
    pieceScores = list(dict.fromkeys(pieceScores))
    return pieceScores
        

In [ ]:
def getRank(pieceScores,gt,idt,queryid,benchmark, condition):
    query = queryid.split('_')[0]
    l = []
    count = 0
    rank = 300000
    for i in range(len(pieceScores)):
        if pieceScores[i][0][0]=='p':
            if(pieceScores[i][0]==queryid.split('_')[0]):
                rank=count+1
                return rank
            if not pieceScores[i][0] in idt.keys():
                count+=1
            elif not idt[pieceScores[i][0]] in l:
                count+=1
                l.append(idt[idt[pieceScores[i][0]]])
        else:
            try:
                if (int(pieceScores[i][0].split("_")[-1]) in gt[query]):
                    rank = count+1
                    return rank

                if not str(int(pieceScores[i][0].split("_")[-1])) in idt.keys():
                    count+=1
                elif not idt[str(int(pieceScores[i][0].split("_")[-1]))] in l:
                    count+=1
                    l.append(idt[str(int(pieceScores[i][0].split("_")[-1]))])
            except:
                pass
    return rank

In [ ]:
def calcPrecisionRecall(hypdir, gtfile, benchmark = False, condition = 1):
    """Inputs: a directory containing the ranked list of pieces from the marketplace fingerprinting system,
               and a file containing the ground truth mapping for each query and the correct IMSLP piece.
               
       Outputs: the MRR for all the predictions, a list of ranks for each query, a list of runtimes for 
                each query, and the number of matches processed for each query."""
    hyps = readHypothesisFiles(hypdir, benchmark)
    gt,idt = readGroundTruthLabels(gtfile)
    MRR = 0
    runtimes = []
    MRRs = []
    matches = []
    hyps = np.array(sorted(hyps, key = lambda x: int(x[0].split("_")[0][1:])*100+int(x[0].split("_")[1][1:])), dtype=object)
    count = 0
    if hyps.shape[1] == 2:
        for queryid, pieceScores in hyps:
            rank = getRank(pieceScores,gt,idt,queryid,benchmark, condition)
            MRRs.append((queryid,rank))
            MRR=MRR+1/(rank)
    else:
        for queryid, pieceScores, runtime, matches_processed in hyps:
            rank = getRank(pieceScores,gt,idt,queryid,benchmark, condition)
            MRRs.append((queryid,rank))
            runtimes.append(runtime)
            MRR=MRR+1/(rank)
            matches.append(matches_processed)
    return MRR/len(MRRs),MRRs, runtimes, matches

In [ ]:
def calcOverlap(seg1, seg2):
    overlap_lb = max(seg1[0], seg2[0])
    overlap_ub = min(seg1[1], seg2[1])
    overlap = np.clip(overlap_ub - overlap_lb, 0, None)
    return overlap    

In [ ]:
hypdir = 'experiments/v0.4.0d_1k_budget/hyp'
benchmark = 0
queryGTFile = 'piece_To_id.csv'
correct_matches = {}
MRR, MRRs, runtimes, matches_processed = calcPrecisionRecall(hypdir, queryGTFile, benchmark, condition = 1)

In [ ]:
MRR

### Investigate Errors

This prints the individual ranks for each query.

In [ ]:
def printDebuggingInfo(MRRs):
    for i, (queryid, rank) in enumerate(MRRs):
        print(queryid, rank)

In [ ]:
printDebuggingInfo(MRRs)

### Measure Runtime

In [ ]:
def showRuntimeStats(durs):
    durs = np.array(durs)
    avgDur = np.mean(durs)
    minDur = np.min(durs)
    maxDur = np.max(durs)
    stdDur = np.std(durs)
    print('Avg runtime: {:.2f} sec'.format(avgDur))
    print('Std runtime: {:.2f} sec'.format(stdDur))
    print('Min runtime: {:.2f} sec'.format(minDur))
    print('Max runtime: {:.2f} sec'.format(maxDur))
    plt.hist(durs, bins=np.arange(0,2,.1))
    plt.xlabel('Runtime (sec)')
    plt.ylabel('Count')

In [ ]:
showRuntimeStats(runtimes)

### Measure matches processed

In [ ]:
def showMatchStats(matchesProcessed):
    matches = np.array(matchesProcessed)
    avgMatches = np.mean(matches)
    minMatches = np.min(matches)
    maxMatches = np.max(matches)
    stdMatches = np.std(matches)
    print(f'Avg matches: {avgMatches}')
    print(f'Std matches: {stdMatches}')
    print(f'Min matches: {minMatches}')
    print(f'Max matches: {maxMatches}')
    plt.hist(matches)
    plt.xlabel('Number of matches processed')
    plt.ylabel('Count')

In [ ]:
showMatchStats(matches_processed)